In [5]:
import keras
from keras.layers import Dense , Flatten , MaxPooling2D , Conv2D , Input
from keras.models import Sequential,load_model , Model
from keras.losses import SparseCategoricalCrossentropy
from keras.activations import relu
from keras.optimizers import Adam
from keras.applications import Xception
from keras.preprocessing.image import load_img , array_to_img , ImageDataGenerator

In [6]:
!pip install kaggle


In [4]:
from google.colab import files
upload = files.upload()

Saving kaggle.json to kaggle.json


In [7]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [8]:
!kaggle datasets download -d omkarjc27/cricket-players-faces

 85% 41.0M/48.0M [00:01<00:00, 42.5MB/s]
100% 48.0M/48.0M [00:01<00:00, 47.7MB/s]


In [9]:
!unzip -q cricket-players-faces

In [10]:
dataPath = '/content/images'
img_height = 224
img_width = 224
dataGen = ImageDataGenerator(
    rescale=1/255.,
    validation_split = 0.2
)
TrainData = dataGen.flow_from_directory(
    dataPath,
    target_size=((img_height, img_width)),
    class_mode='categorical',
    batch_size=32,
    subset='training',
)
validationData = dataGen.flow_from_directory(
    dataPath,
    target_size=((img_height, img_width)),
    class_mode='categorical',
    batch_size=32,
    subset='validation',
)

Found 818 images belonging to 67 classes.
Found 177 images belonging to 67 classes.


In [11]:
baseModel = Xception(include_top=False, weights='imagenet', input_shape=((img_height, img_width, 3)))

for layer in baseModel.layers:
  layer.trainable = False

input_layer = Input(shape=(224,224,3))
modelOuput = baseModel(input_layer) # Pass the input through the VGG16 base model
fOutput = Flatten()(modelOuput) # Flatten the output and add additional fully connected layers
dense_1 = Dense(512, activation='relu')(fOutput)
dense2 = Dense(256, activation='relu')(dense_1)
output_layer = Dense(67, activation='softmax')(dense2)


83683744/83683744 [==============================] - 1s 0us/step


In [12]:
model = Model(inputs=input_layer, outputs=output_layer)
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 512)               51380736  
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 67)                17219     
                                                                 
Total params: 72390763 (276.15 MB)
Trainable params: 51529283

In [ ]:
history = model.fit(
        TrainData,
        epochs = 50,
        validation_data =validationData
)

Epoch 1/50
26/26 [==============================] - 324s 12s/step - loss: 11.5492 - accuracy: 0.0196 - val_loss: 6.2374 - val_accuracy: 0.0452
Epoch 2/50
26/26 [==============================] - 301s 12s/step - loss: 3.4303 - accuracy: 0.2518 - val_loss: 4.4781 - val_accuracy: 0.0847
Epoch 3/50
26/26 [==============================] - 252s 10s/step - loss: 1.6939 - accuracy: 0.6015 - val_loss: 4.4959 - val_accuracy: 0.1299
Epoch 4/50
26/26 [==============================] - 260s 10s/step - loss: 0.7663 - accuracy: 0.8435 - val_loss: 4.6266 - val_accuracy: 0.1412
Epoch 5/50
26/26 [==============================] - 248s 10s/step - loss: 0.3520 - accuracy: 0.9377 - val_loss: 4.8444 - val_accuracy: 0.1751
Epoch 6/50
26/26 [==============================] - 256s 10s/step - loss: 0.1566 - accuracy: 0.9804 - val_loss: 4.7922 - val_accuracy: 0.1695
Epoch 7/50
26/26 [==============================] - 250s 10s/step - loss: 0.0609 - accuracy: 0.9976 - val_loss: 4.9893 - val_accuracy: 0.1695
Epoch